In [1]:
import xarray as xr
import pathlib as pl
import pandas as pd
import pywatershed
import os
import dask
import numpy as np

In [2]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [3]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [4]:
var_output_files = ['hru_actet.nc', 'recharge.nc', 'soil_rechr.nc', 'snowcov_area.nc', 'seg_outflow.nc',]#output files of interest

### Working currently from a single cutout directory

In [5]:
cm = all_models[0] # sets cutout from list
outvardir = rootdir/ cm / 'output'# stes path to location of NHM output folder where output files are.

In [6]:
# set the file name for the postprocessed model output file that PEST will read
of_name = 'alloutput.dat'# name of file

In [7]:
########### Useful code
#indat
#df.index.dtypes
#xr.__version__

### Actual ET
#### Get and check the daily data

In [8]:
actet_daily = xr.open_dataset(outvardir / 'hru_actet.nc')['hru_actet']

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [9]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
actet_monthly = actet_daily.resample(time = 'm').mean()

In [10]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
actet_mean_monthly = actet_monthly.groupby('time.month').mean()

In [11]:
### Used for plotting/checking
#df = actet_monthly.to_dataframe()# makes a dataframe for plotting and writing out
#df_r = np.ravel(actet_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing
#df[8465:8473]
#df.xs(5625, level=1).plot()

#### Now write values to the template file

In [12]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly.indexes['nhm_id']]# set up the indices in sequence
varvals = np.ravel(actet_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='w') as ofp:
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [13]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly.indexes['nhm_id']]
varvals =  np.ravel(actet_mean_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process recharge for calibration use
#### Get daily output file from NHM for recharge

In [14]:
recharge_daily = xr.open_dataset(outvardir / 'recharge.nc')['recharge']

#### Post-process daily output to match observation target of "annual recharge" as an average daily rate for the year

In [15]:
recharge_annual = recharge_daily.resample(time = 'Y').mean()

In [16]:
recharge_annual

<xarray.Dataset>
Dimensions:   (nhm_id: 17, time: 43)
Coordinates:
  * nhm_id    (nhm_id) int32 5621 5625 5628 5635 5637 ... 5728 7128 7156 7157
  * time      (time) datetime64[ns] 1979-12-31 1980-12-31 ... 2021-12-31
Data variables:
    recharge  (time, nhm_id) float64 dask.array<chunksize=(1, 17), meta=np.ndarray>
Attributes:
    Description:  PYNHM output data

#### Write values to template file

In [17]:
inds = [f'{i.year}:{j}' for i in recharge_annual.indexes['time'] for j in recharge_annual.indexes['nhm_id']]
varvals =  np.ravel(recharge_annual.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'recharge_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "soil_rechr" to compare to target
#### Get daily output file from NHM for soil recharge

In [18]:
soil_rechr_daily = xr.open_dataset(outvardir / 'soil_rechr.nc')['soil_rechr']# do this for all

In [19]:
#Creates a dataframe time series of monthly values (average daily rate for each month)
soil_rechr_monthly = soil_rechr_daily.resample(time = 'm').mean() 

#Creates a dataframe time series of annual values (average daily value for each year)
soil_rechr_annual = soil_rechr_daily.resample(time = 'Y').mean() 

In [20]:
inds = [f'{i.year}_{i.month}:{j}' for i in soil_rechr_monthly.indexes['time'] for j in soil_rechr_monthly.indexes['nhm_id']]
varvals = np.ravel(soil_rechr_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_rechr_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [21]:
inds = [f'{i.year}:{j}' for i in soil_rechr_annual.indexes['time'] for j in soil_rechr_annual.indexes['nhm_id']]
varvals =  np.ravel(soil_rechr_annual.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_rechr_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "hru_outflow" to compare to target
#### Get and check the daily data

In [50]:
# These units are in cubic feet (implied per day)
gwres_flow_daily = xr.open_dataset(outvardir / 'gwres_flow_vol.nc')['gwres_flow_vol'] #should be 'hru_outflow, but using a stand-in for now'
sroff_daily = xr.open_dataset(outvardir / 'sroff_vol.nc')['sroff_vol'] #should be 'hru_outflow, but using a stand-in for now'
ssres_flow_daily = xr.open_dataset(outvardir / 'ssres_flow_vol.nc')['ssres_flow_vol'] #should be 'hru_outflow, but using a stand-in for now'

In [51]:
display(gwres_flow_daily)
display(sroff_daily)
display(ssres_flow_daily)

<xarray.DataArray 'gwres_flow_vol' (time: 15706, nhm_id: 17)>
[267002 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2021-12-31
  * nhm_id   (nhm_id) int32 5621 5625 5628 5635 5637 ... 5728 7128 7156 7157
Attributes:
    desc:          Groundwater discharge volume from each GWR to the stream n...
    dims:          ngw
    type:          float64
    units:         cubicfeet
    var_category:  mass flux

<xarray.DataArray 'sroff_vol' (time: 15706, nhm_id: 17)>
[267002 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2021-12-31
  * nhm_id   (nhm_id) int32 5621 5625 5628 5635 5637 ... 5728 7128 7156 7157
Attributes:
    desc:          Surface runoff to the stream network for each HRU
    dims:          nhru
    type:          float64
    units:         cubicfeet
    var_category:  mass flux

<xarray.DataArray 'ssres_flow_vol' (time: 15706, nhm_id: 17)>
[267002 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2021-12-31
  * nhm_id   (nhm_id) int32 5621 5625 5628 5635 5637 ... 5728 7128 7156 7157
Attributes:
    desc:     Interflow volume from gravity and preferential-flow reservoirs ...
    dims:     nssr
    type:     float64
    units:    cubicfeet

In [52]:
# This returns an average daily rate for each month
gwres_flow_monthly = gwres_flow_daily.resample(time = 'm').mean() 
sroff_monthly = sroff_daily.resample(time = 'm').mean() 
ssres_flow_monthly = ssres_flow_daily.resample(time = 'm').mean() 

In [54]:
#ssres_flow_monthly

In [55]:
#This converts the average dailt rate to a rate in cubic feet per second to compare to observation

#hru_outflow_rate = ((gwres_flow_monthly+sroff_monthly+ssres_flow_monthly)/(24*60*60)) #
hru_outflow_rate = gwres_flow_monthly + sroff_monthly + ssres_flow_monthly



In [56]:
hru_outflow_rate

<xarray.DataArray (time: 516, nhm_id: 17)>
array([[ 396966.54283984,   44475.09607433,  959584.5753247 , ...,
        2466638.84965194, 3865877.05384861, 6577906.30146481],
       [ 275388.94219648,   39675.15834137,  778916.12948704, ...,
        1331204.21611217, 2382625.88424138, 7703174.85277799],
       [ 157643.23042109,   25389.0338166 ,  295852.2288791 , ...,
        1031290.44061462, 1698512.02612779, 5165124.5629367 ],
       ...,
       [ 113639.9258788 ,    9281.24762173,   83271.81878836, ...,
        1217413.16639853,  907415.23418421, 2072035.4038751 ],
       [  93456.69193081,    9265.98982471,   60691.85732429, ...,
         666503.47376891, 1017061.0278292 , 2110896.47843714],
       [  82236.35290251,   10346.58602086,   69849.11788562, ...,
         613562.58425899,  805228.36439013, 1588840.89059324]])
Coordinates:
  * nhm_id   (nhm_id) int32 5621 5625 5628 5635 5637 ... 5728 7128 7156 7157
  * time     (time) datetime64[ns] 1979-01-31 1979-02-28 ... 2021-12-31

In [58]:
inds = [f'{i.year}_{i.month}:{j}' for i in hru_outflow_rate.indexes['time'] for j in hru_outflow_rate.indexes['nhm_id']]
varvals = np.ravel(hru_outflow_rate, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'runoff_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "snowcov_area" to compare to target
#### Get and check the daily data

In [ ]:
snowcov_area_daily = xr.open_dataset(outvardir / 'snowcov_area.nc')['snowcov_area']

In [ ]:
inds = [f'{i.year}_{i.month}_{i.day}:{j}' for i in snowcov_area_daily.indexes['time'] for j in snowcov_area_daily.indexes['nhm_id']]
varvals = np.ravel(snowcov_area_daily.to_array(), order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'sca_daily:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "seg_outflow" to compare to target
#### Get and check the daily data

### Create model output template file

In [ ]:
oftemplate_name = 'alloutput_template.dat'

#### AET

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly.indexes['nhm_id']]# set up the indices in sequence
varvals = np.ravel(actet_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / oftemplate_name, encoding="utf-8", mode='w') as ofp:
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly.indexes['nhm_id']]
varvals =  np.ravel(actet_mean_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / oftemplate_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]